<a href="https://colab.research.google.com/github/trishnat/deep_learning1/blob/main/AF3_Ai_class_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predicting Protein-Protein Interactions between Xyllela fastidiosa and Grapes Using the AF3 Server and Hipergator**

We will use the proteome of Xyllela fastidiosa and proteins of Grapes from the literature to study the interactions between these proteins during Pierce's disease in grape.

In [1]:
import json, os
import datetime as dt

### Define a function to prepare json file containing protein-protein pairs

In [ ]:
def fa_to_dict(fa_path):

    """
    convert a fasta file into a python dictionary
    input: fasta file
    output: python dict
    """

    # initialize some storages
    file_items = [] # empty list to store all items read from a fasta file, for easy checking the end of the file
    seqNMs = [] # empty list to store sequence headers
    seqs = [] # empty list to store sequences
    seq = "" # empty string to concatinate all lines of a sequence

    # open fasta file
    with open(fa_path, "r") as file:
        for line in file:
            line = line.strip() # remove the "\n" empty line
            file_items.append(line) # collect all none-empty lines

    for i in range(len(file_items)): # using index to easily check the end of the list

        # if the line starts with ">", this line is a header
        if file_items[i][:1] == ">":
            # remove ">", "SPACE" and "|", and replace "." to "_" for the case of alphafoldserver.com
            seqNMs.append(file_items[i][1:].split(" ")[0].split("|")[0].replace(".", "_"))

            # store the previous sequence
            if seq: # avoid saving the empty string when i=0 at the first header
                seqs.append(seq)
            seq = ""

        # if the line does not start with ">", this line is a sequence line
        else:
            seq = seq + file_items[i]

        # if i reached the end, append the last seq, which couldn't be appended during the first if statement
        if i == len(file_items) -1 :
            seqs.append(seq)

    # convert two lists to a dictionary
    dic = dict(zip(seqNMs, seqs))

    return dic


In [ ]:
def prep_json(fa1_path, fa2_path=None, n=30, out_dir="./"):

    """
    prepare json files that contains pairs of 2 proteins for AF3 on both alphafoldserver.com and HiPerGator

    4 input parameters:
        1) one fasta file for all possible combinations of pair of 2 proteins
        2) or two fasta files for all vs all pairing of 2 proteins
        3) n is the number of pairs in a json file, it is recommended as 30 for alphafoldserver.com, <300 for HiPerGator due to the GPU time limitation (14 days)
        4) out_dir is where you want to save the prepared json files

    output: json files. Each json file has at most n pairs of 2-proteins.
    """
    # get the protein dictionary of fasta file1
    proteins1 = fa_to_dict(fa1_path)
    proteins1_keys = list(proteins1.keys())

    # get the stems of fasta file names for naming json files
    fa1_stem = fa1_path.split("/")[-1].rsplit(".", 1)[0] # rsplit(separator, maxsplit). maxsplit=1, will return a list with 2 elements!
    fa2_stem = ""

    # if fasta file2 is avaiable
    if fa2_path:
        proteins2 = fa_to_dict(fa2_path)
        proteins2_keys = list(proteins2.keys())
        fa2_stem = fa2_path.split("/")[-1].rsplit(".", 1)[0]

    # helpers
    data = [] # empty list to store final results
    counter = 0 # determine when to save a json file


    # n pairs of 2 proteins in a json file
    for i in range(len(proteins1)):

        if fa2_path:
            start = 0
            end = len(proteins2)
            prt2 = proteins2
            prt2_keys = proteins2_keys
        else:
            start = i
            end = len(proteins1)
            prt2 = proteins1
            prt2_keys = proteins1_keys

        # 3 updates
        for j in range(start, end):

            counter += 1

            # create a template dictionary to store protein-protein pair
            temp = {'name': 'protein1_protein2',
                    'modelSeeds': [2025],
                    'sequences': [{'proteinChain': {'sequence': 'ATCG', 'count': 1}},
                                  {'proteinChain': {'sequence': 'ATCG', 'count': 1}}
                                 ]
                   }

            # update job name
            temp["name"]= proteins1_keys[i] + "--vs--" + prt2_keys[j]

            # update the first protein
            temp["sequences"][0]['proteinChain']['sequence'] = proteins1[proteins1_keys[i]]

            # update the second protein
            temp["sequences"][1]['proteinChain']['sequence'] = prt2[prt2_keys[j]]

            # add to the final list
            data.append(temp)


            # Going to save json file with all items been updated
            # get the date today for dir and joson file name
            today = dt.date.today().strftime("%Y%m%d")

            # create a folder if not exist
            new_dir = os.path.join(out_dir, f"{today}_af3_jsons")
            os.makedirs(new_dir, exist_ok=True)

            # if the data list stored n items, save as a json file
            if len(data) == n:
                path = f"{new_dir}/{today}_{fa1_stem}_{fa2_stem}-{counter//n}.json"
                with open(path, "w") as jsonFile:
                    json.dump(data, jsonFile, indent=2)
                data = [] # reset the result list!

            # elif i and j reached the ends
            elif (i == len(proteins1) - 1) & (j == end - 1):
                if data:
                    path = f"{new_dir}/{today}_{fa1_stem}_{fa2_stem}-{counter//n + 1}.json"
                    with open(path, "w") as jsonFile:
                        json.dump(data, jsonFile, indent=2)

### download proteins

In [ ]:
# First install gdown (if not already installed)
!pip install -q gdown

# File ID (from your shared link)
file_id = "1DxjbVk8vFxYd48YpMtHFO2VYydr76ai_"

# Construct the direct download URL
url = f"https://drive.google.com/uc?id={file_id}"

# Download the fa file
!gdown {url}

Downloading...
From: https://drive.google.com/uc?id=1DxjbVk8vFxYd48YpMtHFO2VYydr76ai_
To: /content/proteins.fa
100% 5.92k/5.92k [00:00<00:00, 18.1MB/s]


### Run the function

In [ ]:
prep_json("/content/proteins.fa")

## You have your .json file now its time to run these predictions on the AF3 server.

Go to this link to access the AF3 server on google:

https://alphafoldserver.com/welcome

Sign in with your google account. There will be an option to upload a .json file. After you have downloaded one of the .json files from earlier in this notebook, add it when prompted. This will submit jobs as drafts. You will need to individually run each draft as a job. Once all jobs are complete, select all of the prediction pairs and download a zip file so we can analyze the results.

In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 35.7 MB/s eta 0:00:00


In [ ]:
import json, os, glob, argparse
import pandas as pd
import datetime as dt
import zipfile, io, statistics
from Bio.PDB.MMCIF2Dict import MMCIF2Dict

In [ ]:
def get_args():
    parser = argparse.ArgumentParser("Pipeline to summarize AlphaFold3 results from the zip files that are downloaded from alphafoldserver.com")

    parser.add_argument( "--af3_zf_dir", type=str, required=True, help="The directiory contains running result zip files downloaded from alphafoldserver.com")
    parser.add_argument( "--summary_path", type=str, default="AF3Server_results_summary.csv", help="a csv file path to save the summarized AF3 results")

    args = parser.parse_args()
    return args

In [ ]:
def zip_to_json(zip_file_path):
    """
    input: zip_file_path. Note: one zip file may have multiple AF3 results
    output: a list of 4-element tuples
            a tuple example (proteinNM, js_data_with_ipTM-pTM, [prot1_len, prot2_len], pae_js_data)
    """

    js_lst = [] # one zip file may have multiple AF3 results
    with zipfile.ZipFile(zip_file_path, 'r') as zf:
        # There are many files within a zipfile, iterate them!
        for file_name in zf.namelist():
            # get the "*summary_confidences_0.json" file
            if file_name[-18:] == "confidences_0.json":
                print(file_name)

                # extract the stem for protein names, cif, and data_json
                stem = file_name.split("/")[-1].split("_summ")[0]
                # extract the parent dir for cif, and data_json
                parent_dir = os.path.dirname(file_name)

                # ====read josn summary====
                # protein Names
                proteins = stem.split("fold_")[1]
                # Open this JSON file from the archive
                with zf.open(file_name) as jsf:
                    # Read as JSON data
                    js_data = json.load(jsf)

                # ====read cif file and get the protein lengths====
                # cif path
                cif_file_name_in_zip = os.path.join(parent_dir, f"{stem}_model_0.cif")
                with zf.open(cif_file_name_in_zip) as cif_in_zip:
                    cif_content = cif_in_zip.read().decode("utf-8")  # decode bytes to string
                    cif_dict = MMCIF2Dict(io.StringIO(cif_content))  # wrap in StringIO
                    entity_id = cif_dict['_entity_poly_seq.entity_id']
                    prot1_len = entity_id.count("1")
                    prot2_len = entity_id.count("2")

                # ====read json with PAEs====
                # data json with PAE values
                pae_json_name = os.path.join(parent_dir, f"{stem}_full_data_0.json")
                with zf.open(pae_json_name) as pjsf:
                    # Read as JSON data
                    pae_js_data = json.load(pjsf)

                # ====wrap four items as a tuple====
                tupe = (proteins, js_data, [prot1_len, prot2_len], pae_js_data)

                # add to the the list
                js_lst.append(tupe)

    return js_lst

In [ ]:
def get_avg_pae(prot1_len, prot2_len, pae_json):

    paes = pae_json["pae"] # a list of lists

    # get the one dimension paes for quadrant III
    ## ----vertical line----
    q_iii = []
    for i in range(prot1_len): # quadrant III
        pae = paes[i][prot1_len:] # pae is a sublist of the full list paes[i]
        pae_avg = statistics.mean(pae) # using the avg to represent the pae list
        q_iii.append(pae_avg)

    # get the one dimension paes for quadrant I
    q_i = []
    for i in range(prot1_len, len(paes)): # quadrant I
        pae = paes[i][:prot1_len]
        pae_avg = statistics.mean(pae)
        q_i.append(pae_avg)

    ## ----horizential line----
    # q_iii = []
    # for i in range(prot1_len): # quadrant III
    #     pae = paes[i][:prot1_len] # pae is a sublist of the full list paes[i]
    #     pae_avg = statistics.mean(pae) # using the avg to represent the pae list
    #     q_iii.append(pae_avg)

    # q_i = []
    # for i in range(prot1_len, len(paes)): # quadrant I
    #     pae = paes[i][prot1_len:]
    #     pae_avg = statistics.mean(pae)
    #     q_i.append(pae_avg)

    # get the one number pae for quadrants I and III
    q3_avg = statistics.mean(q_iii) # average of quadrant III's PAEs
    q1_avg = statistics.mean(q_i) # average of quadrant I's PAEs

    # get the final one average pae value
    q13_avg = round((q3_avg + q1_avg)/2, 2) # average of quadrants I and III's PAEs

    return q13_avg

In [ ]:
def main_get_zf_res(af3_zf_dir, save_csv_path):

    # get all .zip files' paths as a list
    zip_path = os.path.join(af3_zf_dir, 'fold*.zip')
    zip_files = glob.glob(zip_path, recursive=True)

    # an empty list to store all lists of iptm and ptm scores
    results = []

    for zf_path in zip_files:

        # an empty list to store iptm and ptm scores for each zip_file
        js_lst = zip_to_json(zf_path) # return a list of 4-element tuples

        for tupe in js_lst:
            result =[]

            # unwrap the tupe
            proteinNMs = tupe[0]
            data = tupe[1] # with ipTM, pTM etc
            prot1_len = tupe[2][0]
            prot2_len = tupe[2][1]
            pae_json = tupe[3] # with PAE values that is a list of lists

            # ====add the protein-protein pair name to the result list====
            result.append(proteinNMs)

            # ====add iptm score====
            result.append(data['iptm'])

            # ====add ptm score====
            result.append(data['ptm'])

            # ====add ranking score fetched====
            result.append(data['ranking_score'])

            # ====get and add average pae====
            q13_avg_pae = get_avg_pae(prot1_len, prot2_len, pae_json)
            result.append(q13_avg_pae)


            # add a result(ppnm, iptm, ptm, ranking_score, avgPAE) list to the results list
            results.append(result)

    # convert results to pandas DataFrame
    df = pd.DataFrame(results, columns=["protein_pair_names", "ipTM", "pTM", "ranking_score", "q13_avg_pae"])

    # sort values by the ranking score
    df = df.sort_values(by=["ipTM", "pTM"], ascending=False)

    # save results as csv
    df.to_csv(save_csv_path, index=False)

    return df

In [ ]:
af3_zf_dir = "/content"
save_csv_path = "/content/AF3_summary.csv"

df_summary = main_get_zf_res(af3_zf_dir, save_csv_path)
df_summary.head()

np_396510_1_vs_at5g49880_1/fold_np_396510_1_vs_at5g49880_1_summary_confidences_0.json
aao54046_1_vs_at5g63960/fold_aao54046_1_vs_at5g63960_summary_confidences_0.json
aan08582_1_vs_at5g24280_3/fold_aan08582_1_vs_at5g24280_3_summary_confidences_0.json
egh89219_1_vs_at5g63960/fold_egh89219_1_vs_at5g63960_summary_confidences_0.json
aaz50536_1_vs_at5g49880_1/fold_aaz50536_1_vs_at5g49880_1_summary_confidences_0.json
aao54046_1_vs_at5g24280_3/fold_aao54046_1_vs_at5g24280_3_summary_confidences_0.json
aax51198_1_vs_at5g24280_3/fold_aax51198_1_vs_at5g24280_3_summary_confidences_0.json
ava31271_1_vs_at5g63960/fold_ava31271_1_vs_at5g63960_summary_confidences_0.json
ccc55840_1_vs_at5g24280_3/fold_ccc55840_1_vs_at5g24280_3_summary_confidences_0.json
aap30775_1_vs_at5g63960/fold_aap30775_1_vs_at5g63960_summary_confidences_0.json
np_396510_1_vs_at4g28630_1/fold_np_396510_1_vs_at4g28630_1_summary_confidences_0.json
aap30775_1_vs_at5g24280_3/fold_aap30775_1_vs_at5g24280_3_summary_confidences_0.json
aaa2

,protein_pair_names,ipTM,pTM,ranking_score,q13_avg_pae
16,egh89219_1_vs_at5g24280_3,0.54,0.36,0.53,30.97
14,acu12491_1_vs_at5g63960,0.52,0.67,0.64,28.24
8,ccc55840_1_vs_at5g24280_3,0.47,0.44,0.51,28.21
15,np_396510_1_vs_at5g24280_3,0.42,0.36,0.45,29.62
24,aax51198_1_vs_at5g63960,0.35,0.65,0.43,26.91
